## Install Dependencies

To accelerate the training process, it's is necessary to run parallel processing on our GPU. For this purpose, install the NVIDIA 9.2 cuda library.

In [2]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb
!apt-key add /var/cuda-repo-9–2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-08-12 19:36:43--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-08-12 19:36:44--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?XroxX1lMtyXJcAMTv61xkOchZavsoGHwXkcPUMaHW_6oAHzqVi9z7C4r-GBePClUiV3zGXLxDjofCxYmEprPY5MPON4BqHbAYzf_HRYJTPWpN_OmB

In [3]:
# Check NVIDIA version
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


Install the spacy library and spacy transformer pipeline

In [4]:
!pip install -U spacy
!python -m spacy download en_core_web_trf

     |████████████████████████████████| 6.4 MB 5.8 MB/s 
     |████████████████████████████████| 42 kB 1.6 MB/s 
     |████████████████████████████████| 456 kB 62.0 MB/s 
     |████████████████████████████████| 621 kB 63.2 MB/s 
     |████████████████████████████████| 10.1 MB 58.3 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
2021-08-12 19:39:48.070232: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully

Install PyTorch library

In [5]:
pip install torch==1.7.1+cu92 torchvision==0.8.2+cu92 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 577.3 MB 2.7 kB/s 
     |████████████████████████████████| 12.5 MB 207 kB/s 
     |████████████████████████████████| 7.6 MB 7.3 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.7.1+cu92 which is incompatible.


## Training of the model

`base_config.cfg`: contains all the model training parameters. I created the file via this site: https://spacy.io/usage/training#quickstart

The `init fill-config` command fills in the remaining defaults and stores it in a file called `config_spacy.cfg`

In [9]:
!python -m spacy init fill-config ./base_config.cfg ./config_spacy.cfg

2021-08-12 20:13:59.344299: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Auto-filled config with all values
✔ Saved config
config_spacy.cfg
You can now add your data and train your pipeline:
python -m spacy train config_spacy.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [10]:
!python -m spacy debug data ./config_spacy.cfg

2021-08-12 20:14:06.789649: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

============================ Data file validation ============================
✔ Corpus is loadable
✔ Pipeline can be initialized with data

=============================== Training stats ===============================
Language: en
Training pipeline: tok2vec, ner
620 training docs
156 evaluation docs
✔ No overlap between training and evaluation data
⚠ Low number of examples to train a new pipeline (620)

============================== Vocab & Vectors ==============================
ℹ 39315 total word(s) in the data (4434 unique)
ℹ No word vectors present in the package

========================== Named Entity Recognition ==========================
ℹ 1 label(s)
0 missing value(s) (tokens with '-' label)
⚠ Low number of examples for label 'BOOK_TITLE' (46)
✔ Examples without occurrences available for all labels
✔ No entities consisting of or s

Training step of model:

In [11]:
!python -m spacy train -g 0 ./config_spacy.cfg --output ./

2021-08-12 20:14:37.394615: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-08-12 20:14:40,014] [INFO] Set up nlp object from config
[2021-08-12 20:14:40,024] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-08-12 20:14:40,028] [INFO] Created vocabulary
[2021-08-12 20:14:40,029] [INFO] Finished initializing nlp object
[2021-08-12 20:14:51,088] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     12.33    0.00    0.00    0.00    0.00
  0     200          9.25    604.87    0.00    0.00    0.00    0.00

-> Those results are a lot lower than previous runs that I did

In [12]:
# zip the best-model after training
!zip -r /model-best.zip ./model-best

  adding: model-best/ (stored 0%)
  adding: model-best/tok2vec/ (stored 0%)
  adding: model-best/tok2vec/model (deflated 8%)
  adding: model-best/tok2vec/cfg (stored 0%)
  adding: model-best/ner/ (stored 0%)
  adding: model-best/ner/model (deflated 7%)
  adding: model-best/ner/cfg (deflated 33%)
  adding: model-best/ner/moves (deflated 46%)
  adding: model-best/tokenizer (deflated 80%)
  adding: model-best/config.cfg (deflated 61%)
  adding: model-best/meta.json (deflated 56%)
  adding: model-best/vocab/ (stored 0%)
  adding: model-best/vocab/strings.json (deflated 73%)
  adding: model-best/vocab/key2row (stored 0%)
  adding: model-best/vocab/vectors (deflated 45%)
  adding: model-best/vocab/lookups.bin (stored 0%)


In [13]:
from google.colab import files
files.download("/model-best.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Load model and extract book information from one podcast

In [14]:
SPACY_MODEL = "./model-best"      # path to spaCy model trained on transformer language model
DATA_FILE = "./episode_115.txt"      # some random Podcast Episode from the Lex Fridman Podcast
ENTITY_LABEL = "BOOK_TITLE"           # label of entity to count

In [15]:
import spacy
nlp = spacy.load(SPACY_MODEL)

### Preprocess the podcast episode
So that the model only runs over sentences that are book title related

In [16]:
# try to preprocess the podcast episode first and only keep sentences with book related words
import spacy
from spacy.matcher import PhraseMatcher

phrase_matcher = PhraseMatcher(nlp.vocab)
phrases = ['book', 'books', 'i read', 'everyone should read', 'you should read', 'he wrote a novel']
patterns = [nlp(text) for text in phrases]
phrase_matcher.add('BOOK_NAME', None, *patterns)

text_with_book_context = []

with open(DATA_FILE) as f:
    sentences_list = [line for line in f]
    
    for sent_i, sent in enumerate(sentences_list):
        for match_id, start, end in phrase_matcher(nlp(sent)):
            if nlp.vocab.strings[match_id] in ["BOOK_NAME"]:
                print(sent)
                text_with_book_context.append(sent)
                break
    print(text_with_book_context)

It really is the best way to support this podcast. Let me say as a side note that Ernest Becker's book, Denial of Death, had a big impact on my thinking about human cognition, consciousness and the deep ocean currents of our mind that are behind the surface behaviors we observe. Many people have told me that they think about death or don't think about death, fear, death, or don't fear death. But I think not many people think about this topic deeply, rigorously in the way that Nietzsche suggested this topic.

It's the best way to support this podcast. This episode is supported by Blankest, my favorite app for Learning New Things Get a Blink is that councillor's legs for seven day free trial and twenty five percent off after Blankest takes the kids from thousands of nonfiction books and condense them down into just 15 minutes that you can read or listen to. I'm a big believer in reading at least an hour a day. As part of that I use Blankest every day. And in general, it's a great way to 

### Run NER Model on this text and extract all book titles


In [17]:
book_titles = []

for doc in nlp.pipe(text_with_book_context):
    for ent in doc.ents:
        if ent.label_ == ENTITY_LABEL:
            if ent.text not in book_titles:
                book_titles.append(ent.text)
            print("Document text: ", doc.text)
            print("Book Title: ", ent.text)

Document text:  It really is the best way to support this podcast. Let me say as a side note that Ernest Becker's book, Denial of Death, had a big impact on my thinking about human cognition, consciousness and the deep ocean currents of our mind that are behind the surface behaviors we observe. Many people have told me that they think about death or don't think about death, fear, death, or don't fear death. But I think not many people think about this topic deeply, rigorously in the way that Nietzsche suggested this topic.

Book Title:  , Denial
Document text:  He died in 1974 after weeks before actually he was posthumously awarded the Pulitzer Prize in nonfiction for his book The Denial of Death.

Book Title:  The Denial of Death
Document text:  And that was his last book. It's actually his next to last book. I don't know how you pulled this off, but he had one more after he died called Escape from Evil. And evidently it was supposed to originally the denial of death was supposed to b

In [18]:
book_titles

[', Denial',
 'The Denial of Death',
 'Escape from Evil',
 'Jared Diamond',
 'Hitler',
 "Darwin's Cathedral",
 'In the Name',
 'Regaining Consciousness',
 'Escape']

-> Also a lot less book titles than previous versions of the model

### Save model on Google Drive
because the download on Colab doesn't work that well

In [ ]:
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

auth.authenticate_user()

In [ ]:
drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
    file_metadata = {
      'name': name,
      'mimeType': 'application/octet-stream'
     }

    media = MediaFileUpload(path, 
                    mimetype='application/octet-stream',
                    resumable=True)

    created = drive_service.files().create(body=file_metadata,
                                  media_body=media,
                                  fields='id').execute()

    print('File ID: {}'.format(created.get('id')))

    return created

In [ ]:
save_file_to_drive("./Google_Colab_Models", "/model-best.zip")

File ID: 1EtBG7ONqPmB395j37h-F3h0jKtCBRS_z


{'id': '1EtBG7ONqPmB395j37h-F3h0jKtCBRS_z'}